# Homework #2
**Due:** Thursday January 18th, 11:59PM via Blackboard

**Name:** *Student doesn't know Markdown but its easy name here*,  **Student Id:** *Your ID here*

## Problem 1. Learning Bridges with Mininet (20 pts)
You will need a running Mininet VM for this problem. See [working with the Mininet VM](http://www.grotto-networking.com/SDNfun.html#working-with-the-mininet-vm) for configuration tips.

**(a)** Mininet is Python based. In fact you can control all aspects of Mininet from Python.  From a terminal to your Mininet VM use the command `sudo python`. This brings up the Python interpreter as *superuser* which is needed since Mininet creates network interfaces and such in the VM. At the Python command line enter the code below to create, run, and test a very simple network:

```python
from mininet.net import Mininet
from mininet.topo import Topo
topo = Topo()  # Create an empty topology
topo.addSwitch("s1")  # Add switches and hosts to the topology
topo.addHost("h1")
topo.addHost("h2")
topo.addLink("h1", "s1") # Wire the switches and hosts together with links
topo.addLink("h2", "s1")
net = Mininet(topo)  # Create the Mininet, start it and try some stuff
net.start()
net.pingAll()
net.iperf()
net.stop()
```

Capture a screen shot of your work. Note that *iperf* performance numbers.  I got something like ![Mininet test1](HW2Mini1.png)

**(b)** Enter the following code into the Python interpreter (`sudo python`) running on your Mininet VM. Does the a *ping* test between hosts work for you? Why or why not? Note that the default controller for the switches in Mininet implements an Ethernet learning bridge but does not support the spanning tree protocol.

```python
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.link import TCLink  # So we can rate limit links
from mininet.cli import CLI  # So we can bring up the Mininet CLI
topo = Topo()  # Create an empty topology
topo.addSwitch("s1")  # Add switches and hosts to the topology
topo.addSwitch("s2")
topo.addSwitch("s3")
topo.addHost("h1")
topo.addHost("h2")
topo.addHost("h3")
# Wire the switches and hosts together. Note there is a loop!
topo.addLink("h1", "s1", bw=20.0, delay='10ms', use_htb=True)
topo.addLink("h2", "s2", bw=25.0, delay='10ms', use_htb=True)
topo.addLink("h3", "s3", bw=25.0, delay='10ms', use_htb=True)
topo.addLink("s1", "s2", bw=11.0, delay='40ms', use_htb=True)
topo.addLink("s1", "s3", bw=15.0, delay='7ms', use_htb=True)
topo.addLink("s2", "s3", bw=5.0, delay='7ms', use_htb=True)
net = Mininet(topo=topo, link=TCLink)
net.start()
CLI(net)  # Bring up the mininet CLI
net.stop()
```

Take a screen shot, I got something like ![Mininet test 2](HW2Mini2.png)



## Problem 2. Tunnels and such (20 pts)

**(a)** Explain how a *ping* (ICMP echo request/response) get through a "NAT" box. Cite any web pages or references that you use.

**(b)** Suppose you are using *GRE* to extend an *Ethernet* network between two sites over the Internet. Draw a rough packet diagram that show the protocol nesting (packets within packets) of an example tunnel packet that would be seen on the internet.  What kind of information would the tunnel end points (TE1 and TE2) need to keep to make this work? ![tunnel](Net2NetTunnel.png). 


## Problem 3. Paths and Trees and Network Creation (20 pts)
**(a)** How many different trees does a linear network (A<-->B<-->C...) consisting of N > 2 nodes have?

**(b)** How many different trees does a ring network consisting of N > 3 nodes have?

**(c)** How many different paths are between two nodes in tree network? Why?

**(d)** Using the [ComNetViz](https://www.grotto-networking.com/ComNetViz/ComNetViz.html) web app create a network with at least 15 switches, but no hosts.  Use **your initials** as the start of the switch names, i.e., for me *GB1, GB2,...*.  Make sure that the switches are connected with lots of links to be able to withstand switch and link failures and to support alternative paths.  When you first create the links you can use any *weight* and *capacity* you wish. After you've finished adding nodes and links to the network use the "Net/Distance Based Weights" menu item to assign weights to all the links. Use the "Net/Randomize Capacities" menu item to assign random capacities to the links between **10 and 200** units. Under the "View" menu item make sure the "Show link capacity" item is checked so that the graphical thickness of the links is related to the links capacity. Export the network into a JSON file (which you will use in the next problem) and include a screen shot of your network here, e.g., 
![Dr. B's example network](HW2ExampleNet.png)

## Problem 4 Shortest Paths, Widest Paths, and Spanning Trees (20 pts)
**(a)** In your network from problem 3(d) choose any node and compute all the shortest paths to that node. Use the "Path/Shortest Paths" menu item with your chosen path as the *source* and "all" as the destination, click "compute", then click "apply". Click on "Path/Paths Info" to bring up the "Paths Info" panel which allows you to add/remove and visualize paths. Click the "All Paths" tab and select all the shortest paths that you just computed.  Do these paths form a tree?  Include a screen shot of your network with the paths highlighted and the paths panel, e.g., ![Shortest paths from A4](HW2PathsShort1.png)

**(b)** In your network from problem 3(d) choose a different node from part (a) and compute all the shortest paths from that node. Do these paths form a tree?  Are these paths the same as in part (a)? Highlight all the paths and capture a screen shot, e.g., ![Shortest paths from C12](HW2PathsShort2.png)

**(c)** In these bandwidth hungry times we might be interested in the "widest paths" (in terms of capacity) from a node.  What limits the capacity of a path? Choose any node from your network then compute the widest paths from that node ("Path/Widest Paths"). Highlight all of these paths. Do these paths for a tree? Is this set of paths the same as in parts (a) or (b)? Capture a screen shot, e.g., ![Widest paths from B4](HW2PathsWide.png).

**(d)** Compute the Minimum spaning tree ("Path/Minimum Spanning Tree"). The app returns all the links of the tree each considered a "path". Is the collection of paths induced by this tree the same as any of those in parts (a)-(c)? Capture a screen shot, e.g., ![Min span tree](HW2PathsMSP.png)

**(e)** How many different trees does your network contain?  Hint see [Kirchhoff's Theorem](https://en.wikipedia.org/wiki/Kirchhoff's_theorem). See below for an example of how I calculated this for my example network.

In [1]:
# How I computed the number of trees in my network
from networkx.readwrite import json_graph
import networkx as nx
import json
import trees  # imports Dr. B's tree computation module.

# Read in a JSON formatted graph file, put your file name in here not mine!
gnl = json.load(open("HW2Prob3Ex.json"))
g = json_graph.node_link_graph(gnl)
# Basic node and link info
print("Network nodes: {}".format(g.nodes()))
# print("Network links: {}".format(g.edges()))
# Compute number of trees
num_trees = trees.exact_num_trees(g)
print("The number of trees in this graph is {}".format(num_trees))

Network nodes: ['B9', 'A17', 'B3', 'C12', 'E20', 'D6', 'B22', 'D30', 'B16', 'D20', 'B28', 'A28', 'E5', 'C17', 'D14', 'E17', 'C29', 'E8', 'A23', 'E28', 'E15', 'A4', 'D16', 'D4', 'A14', 'A9', 'C8', 'B4', 'C6', 'C24']
The number of trees in this graph is 4747921703663.972


## Problem 5 Ethernet VLANs and Multiple Spanning Trees (20 pts)
Suppose that you work for an animation company with 24 employees divided into three teams working on different films.  Since each team member is involved with intensive computing and networking they each have a *wired* workstation with a 10Gbp Ethernet link. Your job will be to assign machines to VLANs based on films and associated VLANs with different spanning trees in a better than random manner.
**(a)** Using ComNetViz create an example company network consists of 5 switches each with (up to 8) 10Gbps links to hosts and (up to three) 100Gbps links between switches. Make sure that each switch is connected to two to three other switches (but no more) for reliability purposes.  Label the hosts for team 1 as H11, H12,...H16, for team 2 H21, H22, ..., H26, and similarly for team three. Randomly assign hosts to switches making sure not to exceed twelve hosts per any one switch. Make when creating hosts that you assign them a VID (VLAN Id).  A smaller example network that I created looks like ![Simple VLAN Net](HW2VLANEx.png)

**(b)** How many trees does your VLAN network support?

**(c)** In **general** is assigning VLANs to trees to optimize bandwidth to individual teams a hard problem? Why?
**(d)** Come up with a reasonable assignment of VLANs to trees. Show me some graphical depiction of the trees. For example below I tried an algorithm where I kept only the members of each VLAN and computed the minimum spanning tree in each case. Did this approach work lead to better use of the inter-switch links? ![VLAN1](HW2VLAN1.png), ![VLAN2](HW2VLAN2.png), ![VLAN3](HW2VLAN3.png), 

In [2]:
# Read in a JSON formatted graph file, put your file name in here not mine!
gnl = json.load(open("DrBVLANEx18.json"))
g = json_graph.node_link_graph(gnl)
# Compute number of trees
num_trees = trees.exact_num_trees(g)
print("The number of trees in this graph is {}".format(num_trees))

The number of trees in this graph is 24.000000000000014
